# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 21/02/2024

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial

In [ ]:
# load one of the first datasets to get the paths

dset_path = '/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/20240724/FeAu_0p5_tR/FeAu_0p5_tR_ff1/FeAu_0p5_tR_ff1_dataset.h5'

# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

print(ds)
print(ds.shape)

rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

In [ ]:
# USER: pick a sample you want to import

skips_dict = {
    "FeAu_0p5_tR": []
}

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]

samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)

print(samples_dict)

In [ ]:
# load all 3DXRD datasets for this sample

from collections import OrderedDict

# just take first sample for now

sample = sample_list[0]
datasets = samples_dict[sample]
ds_dict = OrderedDict()

# try to sort datasets alphabetically

datasets_sorted = sorted(datasets)

for dataset in datasets_sorted:
    dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
    ds = ImageD11.sinograms.dataset.load(dset_path)
    print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
    ds_dict[dataset] = ds

In [ ]:
# populate z translations

z_translation_motor = "samtz"

for ds in ds_dict.values():
    with h5py.File(ds.masterfile, 'r' ) as hin:
        this_z_trans_value = hin["1.1/instrument/positioners"][z_translation_motor][()]
    ds.zpos = this_z_trans_value  # this is in microns for samtz

In [ ]:
# load grains for each dataset and tie them to the dataset objects
for ds in ds_dict.values():
    ds.grains = ds.get_grains_from_disk()

In [ ]:
# get positions within the sample (set middle slice as zero position)

middle_ds = list(ds_dict.values())[len(ds_dict.values())//2]
middle_pos = middle_ds.zpos

for ds in ds_dict.values():
    # adjust so that the first letterbox (lowest z so highest on the sample) has the highest value of zpos
    ds.zpos_sample = middle_pos - ds.zpos
    print(ds.zpos, ds.zpos_sample)

In [ ]:
for inc, ds in enumerate(ds_dict.values()):
    for gid, grain in enumerate(ds.grains):
        grain.pos_sample = grain.translation + np.array([0., 0., ds.zpos_sample * 1000])
        grain.dataset = ds.dsname
        grain.z_slice = inc
        grain.gid = gid

In [ ]:
all_grains = []
for ds in ds_dict.values():
    all_grains.extend(ds.grains)

In [ ]:
print(all_grains[0].pos_sample, all_grains[0].translation)

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

import matplotlib.cm as cm

colors = cm.rainbow(np.linspace(0, 1, len(list(ds_dict.values()))))

for ds in ds_dict.values():
    xx = [grain.pos_sample[0] for grain in ds.grains]
    yy = [grain.pos_sample[1] for grain in ds.grains]
    zz = [grain.pos_sample[2] for grain in ds.grains]
    # col = [len(grain.peaks_3d) for grain in ds.grains]
    # col = [grain.z_slice for grain in ds.grains]
    scatterplot = ax.scatter(xx, yy, zz, c=colors[ds.grains[0].z_slice], label=ds.grains[0].z_slice)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-100,100)
# plt.colorbar(scatterplot)
ax.set_title("Grains coloured by z slice")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.legend()
plt.show()

In [ ]:
# now we can look for duplicate grains
# this is a very simple duplicate grain detector

distance_tolerance = 25/2  # microns
angle_tolerance = 2  # degrees

def eudis(v1, v2):
    return np.linalg.norm(v1-v2)

from xfab.symmetry import Umis

def misorien_deg(U1, U2):
    return np.min(Umis(U1, U2, 7), axis=0)[1]  # 7 == cubic

matches = []

for grain_a in all_grains:
    for grain_b in all_grains:
        if grain_a.z_slice == grain_b.z_slice:
            # grains are in the same slice, skip
            continue
        translation = eudis(grain_a.pos_sample, grain_b.pos_sample)
        if translation < distance_tolerance:
            # might have a match in translation, now check misorientation
            misorien = misorien_deg(grain_a.U, grain_b.U)
            if misorien < angle_tolerance:
                print(f"Found match! Grain A: {grain_a.z_slice}:{grain_a.gid} | Grain B: {grain_b.z_slice}:{grain_b.gid} | Distance: {translation:.0f} um | Angle: {misorien:.3f} deg")
                matches.append((grain_a, grain_b))

In [ ]:
# how do we determine good choices of parameters?
# one way is to follow Louca and try a range of parameters and plot the results

In [ ]:
def get_n_matches(dist_tol, ang_tol):
    n_matches = 0
    for grain_a in all_grains:
        for grain_b in all_grains:
            if grain_a.z_slice == grain_b.z_slice:
                # grains are in the same slice, skip
                continue
            translation = eudis(grain_a.pos_sample, grain_b.pos_sample)
            if translation < dist_tol:
                # might have a match in translation, now check misorientation
                misorien = misorien_deg(grain_a.U, grain_b.U)
                if misorien < ang_tol:
                    n_matches += 1
    return n_matches

In [ ]:
dist_tols = [1, 2, 5, 10, 15, 20, 25]
ang_tols = [1, 2, 3, 4, 5]

tol_check_results = {}

print("dist_tol | ang_tol | n_matches")

for dist_tol in dist_tols:
    for ang_tol in ang_tols:

        n_matches = get_n_matches(dist_tol, ang_tol)
        tol_check_results[dist_tol, ang_tol] = n_matches
        print(f"{dist_tol} | {ang_tol} | {n_matches}")

In [ ]:
fig, ax = plt.subplots()

for ang_tol in ang_tols:
    n_matches_at_this_angle = []
    for dist_tol in dist_tols:
        n_matches_at_this_distance = tol_check_results[dist_tol, ang_tol]
        n_matches_at_this_angle.append(n_matches_at_this_distance)
    
    ax.plot(dist_tols, n_matches_at_this_angle, label=ang_tol)
    
ax.legend()

ax.set_xlabel("Distance tolerance (um)")
ax.set_ylabel("Number of merges")
ax.set_title("Legend is angle tolerance (deg)")

plt.show()

In [ ]:
# please note that this is a very simple duplicate grain detector
# it doesn't consider things like whether grains can match to multiple other grains
# or whether there could be duplicate grains within a single slice
# we're also not currently considering how to merge grains together
# this involves averaging the positions (easy) and the UBIs (not so easy)
# hopefully this notebook gives you a starting point though!